<a href="https://colab.research.google.com/github/laurahsisson/smoker-status/blob/main/Flocking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Binary Prediction of Smoker Status using Bio-Signals
[From Kaggle Playground Series](https://www.kaggle.com/competitions/playground-series-s3e24/overview)

Using flock of single-layer MLPs

In [ ]:
import pandas as pd
import torch
import sklearn
import sklearn.model_selection

In [ ]:
%pip install torchmetrics
import torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 5.6 MB/s eta 0:00:00


In [ ]:
%pip install optuna
import optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')
input_dir = "/content/drive/MyDrive/Data/Playground/"

Mounted at /content/drive


In [ ]:
train = pd.read_csv(os.path.join(input_dir, "train.csv"))
evaluate = pd.read_csv(os.path.join(input_dir, "test.csv"))
sample_submission = pd.read_csv(os.path.join(input_dir, "sample_submission.csv"))

In [ ]:
print(f"Training data of size = {train.shape}")
print(f"Test data of size = {evaluate.shape}")

Training data of size = (159256, 24)
Test data of size = (106171, 23)


In [ ]:
x_columns = train.columns[train.columns != "smoking"]
y_column = train.columns[train.columns == "smoking"]

print("Input data for id = 0")
display(train.iloc[0][x_columns])
print()
print("Target value for id = 0")
display(train.iloc[0][y_column])

Input data for id = 0


id                       0.0
age                     55.0
height(cm)             165.0
weight(kg)              60.0
waist(cm)               81.0
eyesight(left)           0.5
eyesight(right)          0.6
hearing(left)            1.0
hearing(right)           1.0
systolic               135.0
relaxation              87.0
fasting blood sugar     94.0
Cholesterol            172.0
triglyceride           300.0
HDL                     40.0
LDL                     75.0
hemoglobin              16.5
Urine protein            1.0
serum creatinine         1.0
AST                     22.0
ALT                     25.0
Gtp                     27.0
dental caries            0.0
Name: 0, dtype: float64


Target value for id = 0


smoking    1.0
Name: 0, dtype: float64

In [ ]:
# Min max normalzie values
x_train = train[x_columns]
x_train_normalized = (x_train-x_train.min())/(x_train.max()-x_train.min())
x_train_normalized.sample(5)

,id,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,...,triglyceride,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries
48,0.000301,0.615385,0.454545,0.35,0.355263,0.071429,0.061224,0.0,0.0,0.301471,...,0.105541,0.220472,0.053254,0.478261,0.0,0.040816,0.022021,0.004463,0.007021,0.0
135232,0.849154,0.538462,0.454545,0.35,0.398684,0.061224,0.061224,0.0,0.0,0.301471,...,0.156992,0.338583,0.047337,0.658385,0.4,0.102041,0.053109,0.009612,0.027081,0.0
21075,0.132335,0.384615,0.636364,0.45,0.513158,0.091837,0.142857,0.0,0.0,0.500000,...,0.459103,0.314961,0.068854,0.658385,0.2,0.061224,0.025907,0.009612,0.049147,0.0
114675,0.720072,0.307692,0.454545,0.15,0.250000,0.112245,0.112245,0.0,0.0,0.382353,...,0.026385,0.299213,0.048413,0.372671,0.0,0.051020,0.010363,0.002746,0.009027,0.0
47888,0.300700,0.307692,0.636364,0.35,0.355263,0.112245,0.112245,0.0,0.0,0.242647,...,0.193931,0.354331,0.067778,0.708075,0.0,0.112245,0.020725,0.006179,0.012036,0.0


In [ ]:
eval_normalized = (evaluate-x_train.min())/(x_train.max()-x_train.min())
eval_normalized.sample(5)

,id,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,...,triglyceride,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries
56362,1.353917,0.153846,0.727273,0.40,0.394737,0.091837,0.091837,0.0,0.0,0.198529,...,0.089710,0.464567,0.046261,0.739130,0.2,0.102041,0.022021,0.004119,0.021063,0.0
24181,1.151845,0.307692,0.545455,0.35,0.447368,0.091837,0.081633,0.0,0.0,0.301471,...,0.166227,0.275591,0.089833,0.732919,0.0,0.081633,0.023316,0.008239,0.017051,0.0
6173,1.038768,0.307692,0.727273,0.45,0.401316,0.142857,0.112245,0.0,0.0,0.455882,...,0.158311,0.346457,0.087682,0.658385,0.0,0.102041,0.009067,0.002746,0.015045,0.0
88594,1.556309,0.538462,0.272727,0.20,0.221053,0.051020,0.061224,0.0,0.0,0.301471,...,0.065963,0.700787,0.072620,0.534161,0.0,0.040816,0.020725,0.005493,0.013039,0.0
102274,1.642209,0.615385,0.545455,0.30,0.394737,0.081633,0.091837,1.0,1.0,0.441176,...,0.186016,0.393701,0.047875,0.583851,0.0,0.081633,0.016839,0.003776,0.015045,0.0


In [ ]:
train_x_tensor = torch.FloatTensor(x_train_normalized.to_numpy())
train_y_tensor = torch.FloatTensor(train[y_column].to_numpy())
eval_x_tensor = torch.FloatTensor(eval_normalized.to_numpy())

print(f"train_x_tensor = {train_x_tensor.shape}",f"train_y_tensor = {train_y_tensor.shape}",f"eval_x_tensor = {eval_x_tensor.shape}",sep="\n")
print(train_x_tensor.min(),train_x_tensor.max())
print(eval_x_tensor.min(),eval_x_tensor.max())

train_x_tensor = torch.Size([159256, 23])
train_y_tensor = torch.Size([159256, 1])
eval_x_tensor = torch.Size([106171, 23])
tensor(0.) tensor(1.)
tensor(-0.0449) tensor(1.6667)


In [ ]:
def get_device():
    device = "cpu"
    if torch.cuda.is_available():
      device = "cuda:0"
    return device

class SmokingDataset(torch.utils.data.Dataset):
    def __init__(self,data,target=None):
        self.data = data.to(get_device())

        # Null coalesce the target to an empty of the expected shape.
        target = target if torch.is_tensor(target) else torch.empty((len(data),1))
        self.target = target.to(get_device())

    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]

        return {'data': x, 'target': y}

    def __len__(self):
        return len(self.data)

bsz = 512
tr_x, ts_x, tr_y, ts_y = sklearn.model_selection.train_test_split(train_x_tensor, train_y_tensor)
print(f"tr_x = {tr_x.shape}",f"tr_y = {tr_y.shape}",sep=",\t")
print(f"ts_x = {ts_x.shape}",f"ts_y = {ts_y.shape}",sep=",\t")

train_loader = torch.utils.data.DataLoader(SmokingDataset(tr_x,tr_y), batch_size=bsz)
test_loader = torch.utils.data.DataLoader(SmokingDataset(ts_x,ts_y), batch_size=bsz)

example_batch = next(iter(train_loader))
example_batch["data"].shape, example_batch["target"].shape

tr_x = torch.Size([119442, 23]),	tr_y = torch.Size([119442, 1])
ts_x = torch.Size([39814, 23]),	ts_y = torch.Size([39814, 1])


(torch.Size([512, 23]), torch.Size([512, 1]))

In [ ]:
def make_sequential(input_dim, hidden_dim, output_dim, dropout):
  return torch.nn.Sequential(torch.nn.Linear(input_dim, hidden_dim), torch.nn.ReLU(), torch.nn.Dropout(p=dropout), torch.nn.Linear(hidden_dim, output_dim))

def do_epoch(models,train_loader,test_loader):
  model = models["model"]
  opt = models["opt"]
  loss_fn = models["loss_fn"]
  auroc_fn = models["auroc_fn"]

  for batch in train_loader:
    opt.zero_grad()
    fwd = model(batch["data"])
    loss = loss_fn(fwd,batch["target"])
    loss.backward()
    opt.step()

  with torch.no_grad():
    ts_pred = []
    ts_target = []
    for batch in test_loader:
      ts_pred.append(model(batch["data"]))
      ts_target.append(batch["target"])

  return auroc_fn(torch.cat(ts_pred),torch.cat(ts_target)).cpu().numpy()

def get_config(trial):
  return {
    "lr": trial.suggest_float("lr", 1e-10,1e-1,log=True),
    "dropout": trial.suggest_float("dropout", 0,1),
    "hidden_dim": trial.suggest_int("hidden_dim",2,512,log=True),
    "epochs": trial.suggest_int("epochs",1,100),
  }

def make_models(config):
  model = make_sequential(len(x_columns),config["hidden_dim"],1,config["dropout"])
  model.to(get_device())
  return {
    "loss_fn": torch.nn.BCEWithLogitsLoss(),
    "model": model,
    "opt": torch.optim.Adam(model.parameters(),lr=config["lr"]),
    "auroc_fn": torchmetrics.AUROC(task="binary"),
  }

def train_model(trial,display_model=False):
  config = get_config(trial)

  models = make_models(config)
  if display_model:
    display(models["model"])

  for i in range(config["epochs"]):
    loss = do_epoch(models,train_loader,test_loader)

    trial.report(loss, i)
    if i % 10 == 0:
      print(i,loss)
    if trial.should_prune():
      print("Pruning after",i)
      raise optuna.TrialPruned()

  return loss

In [ ]:
example_config = {
    "lr": 1e-3,
    "dropout": .1,
    "hidden_dim": 256,
    "epochs": 5,
    "flock": 3,
}
all_models = []
for _ in range(example_config["flock"]):
  all_models.append(make_models(example_config))


In [ ]:
def flatten(models):
  m = models["model"]
  p1 = [p.flatten() for p in m.parameters()]
  return torch.cat(p1)

def calculate_distances(models):
  with torch.no_grad():
    model_positions = torch.stack([flatten(ms) for ms in all_models])
    dists = torch.cdist(model_positions,model_positions)
    return dists



scores = torch.rand((len(all_models),1))
dists = calculate_distances(all_models)
print(scores)
print(dists)
weights = scores/dists
weights

tensor([[0.7785],
        [0.0681],
        [0.9271]])
tensor([[ 0.0000, 13.2141, 13.3522],
        [13.2141,  0.0000, 13.4409],
        [13.3522, 13.4409,  0.0000]])


tensor([[   inf, 0.0589, 0.0583],
        [0.0052,    inf, 0.0051],
        [0.0694, 0.0690,    inf]])

In [ ]:
model = all_models[0]["model"]
others = [m["model"] for m in all_models]
ps = [[p for p in m.parameters()] for m in others]
idx = 0

for otheridx in range(len(others)):
  if idx == otheridx:
    continue
  print(otheridx)
  weight = weights[otheridx][idx]
  print(weight)
  for p in ps[otheridx]:
    print((p*weight).shape)


1
tensor(0.0052)
torch.Size([256, 23])
torch.Size([256])
torch.Size([1, 256])
torch.Size([1])
2
tensor(0.0694)
torch.Size([256, 23])
torch.Size([256])
torch.Size([1, 256])
torch.Size([1])


In [ ]:

with torch.no_grad():
  for m in model.parameters():
    print(m.flatten()[0])
    m.copy_(m*.5)

  print()
  for m in model.parameters():
    print(m.flatten()[0])



tensor(0.0454, requires_grad=True)
tensor(0.1488, requires_grad=True)
tensor(-0.0524, requires_grad=True)
tensor(-0.0533, requires_grad=True)

tensor(0.0227, requires_grad=True)
tensor(0.0744, requires_grad=True)
tensor(-0.0262, requires_grad=True)
tensor(-0.0266, requires_grad=True)


In [ ]:
params = [[torch.rand((2,2)), torch.rand((1))] for _ in range(3)]
scores = torch.rand(len(params))
# for ps in params:
#   print(ps)
print()
print()
print()
# print(scores)
print()
stacked_ps = [torch.stack([ps[i] for ps in params]) for i in range(len(params[0]))]
centers = []
for sp in stacked_ps:
  dimensions_to_add = (None,)*(len(sp.shape)-1)
  print(sp)
  ts = scores[(...,)+dimensions_to_add]
  weighted_params = sp*ts
  centers.append(weighted_params.sum(dim=0)/ts.sum())

cohesion_steering = []
for i,sp in enumerate(stacked_ps):
  deltas = centers[i].unsqueeze(0)-sp
  cohesion_steering.append(deltas)
  # STEERING FORCE IS delta*forcefactor
  # So you would do weights + delta*ff





tensor([[[0.2813, 0.6285],
         [0.2049, 0.2254]],

        [[0.2404, 0.5768],
         [0.6771, 0.2962]],

        [[0.0630, 0.6555],
         [0.2056, 0.2023]]])
tensor([[0.8480],
        [0.1607],
        [0.1799]])


In [ ]:
gradients = [[torch.rand((2,2)), torch.rand((1))] for _ in range(3)]
gradient_steering = []
stacked_gs = [torch.stack([ps[i] for ps in gradients]) for i in range(len(params[0]))]
for sg in stacked_gs:
  dimensions_to_add = (None,)*(len(sg.shape)-1)
  print("FFF")
  print(sg)
  print(scores)
  ts = scores[(...,)+dimensions_to_add]
  weighted_gradients = sg*ts
  gradient_steering.append(weighted_gradients.sum(dim=0)/ts.sum())
  print(gradient_steering[-1])
  print()

FFF
tensor([[[0.5543, 0.1238],
         [0.4084, 0.7454]],

        [[0.3935, 0.8902],
         [0.8253, 0.5952]],

        [[0.1743, 0.3090],
         [0.0907, 0.7032]]])
tensor([0.0628, 0.8274, 0.0199])
tensor([[0.3998, 0.8246],
        [0.7805, 0.6079]])

FFF
tensor([[0.8363],
        [0.2059],
        [0.5466]])
tensor([0.0628, 0.8274, 0.0199])
tensor([0.2568])



In [ ]:
def stack(all_params):
  return [torch.stack([ps[i] for ps in all_params]) for i in range(len(all_params[0]))]

def weighted_average(stacked_params,scores):
  centers = []
  normscores = scores/scores.sum()
  for sp in stacked_params:
    dimensions_to_add = (None,)*(len(sp.shape)-1)
    ts = normscores[(...,)+dimensions_to_add]
    weighted_params = sp*ts
    centers.append(weighted_params.sum(dim=0))

  return centers

# For a flock of n with param of shape (a,b), returns steering force of shape (a,b)
def align(all_gradients,scores,align_factor):
  stacked = stack(all_gradients)
  steering = weighted_average(stacked,scores)
  return [s*align_factor for s in steering]

# For a flock of n with param of shape (a,b), returns steering force of shape (n,a,b)
# Because each agent must steer differently to reach the center
# We could just return the weighted centers and have each agent steer in its own update step
# but this is more efficent (although flock size << param count)
def cohere(all_params,scores,cohere_factor):
  stacked = stack(all_params)
  centers = weighted_average(stacked,scores)
  steering = []
  for i,sp in enumerate(stacked_ps):
    deltas = centers[i].unsqueeze(0)-sp
    steering.append(deltas)
  return [s*cohere_factor for s in steering]


print(align(gradients,scores,.1))





[tensor([[0.0356, 0.0519],
        [0.0483, 0.0666]]), tensor([0.0465])]


In [ ]:
flatten_ps = [sp.flatten(start_dim=1) for sp in stacked_ps]
positions = torch.cat(flatten_ps,dim=1)
distances = torch.cdist(positions,positions)
# print(distances)
sep_distance = .6
mask_close = distances < sep_distance
# Distances will always be a square because it is distance from every agent to every other agent
mask_not_self = ~torch.eye(distances.shape[0],dtype=torch.bool)
mask_separate = mask_close & mask_not_self
r = distances/sep_distance
# Force is delta / r^2 regardless of score but must include mask.
# Might need to include an EPS so we don't divide by 0



normscores = scores / scores.sum()
for sp in stacked_ps:
  # n x n x a x b
  # Where i, j is sp[i] - sp[j]
  # To steer sp[i] away from sp[j]
  # You can do sp[i] + delta[i][j]
  #
  delta = sp.unsqueeze(1) - sp.unsqueeze(0)
  # print(sp.shape)
  # print(delta.shape)
  # print(r.shape)
  rd = r + .00001
  dimensions_to_add = (None,)*(len(sp.shape)-1)

  trd = rd[(...,)+dimensions_to_add]
  tmsk = mask_separate[(...,)+dimensions_to_add]

  ts = normscores[(...,)+(None,)*(len(sp.shape))]
  delta_scaled = (delta*tmsk*ts)/torch.square(trd)

  # Sum up the individual steering forces.
  # Could do a weighted sum later but I think this is fine.
  # Assuming the gradient is relatively smooth then the closer agents
  # then the repulsive force scales linearly with the gradient.


  steering = delta_scaled.sum(dim=0)
  # print(steering.shape,steering,sep="\n")
  # print(sp-steering)
  for i in range(len(sp)):
    for j in range(len(sp)):
      if i == j:
        continue
      if not mask_separate[i][j]:
        continue
      # print(delta[i][j])
      d = sp[i] - sp[j]
      print(i,j)
      # print("i",sp[i])
      # print("j",sp[j])
      # print("vd",d*normscores[j]/torch.square(trd[j]))
      # print("md",delta_scaled[i][j])
      print(normscores[j])
      print()
      # This means that the bigger we are, the more we are pulled
      # When in reality it should be the opposite.
      print("vd",d*normscores[i]/torch.square(rd[i][j]))
      print("md",delta_scaled[i][j])
      # print()
      # print()
      # print()
      # print()
  break
  print()
  print()
  print()
  print()
  print()
  print()
  # break

1 2
tensor(0.3370)

vd tensor([[ 0.1031, -0.0457],
        [ 0.2741,  0.0546]])
md tensor([[ 0.1031, -0.0457],
        [ 0.2741,  0.0546]])
2 1
tensor(0.4346)

vd tensor([[-0.0799,  0.0354],
        [-0.2125, -0.0423]])
md tensor([[-0.0799,  0.0354],
        [-0.2125, -0.0423]])


In [ ]:
flatten_ps = [sp.flatten(start_dim=1) for sp in stacked_ps]
positions = torch.cat(flatten_ps,dim=1)
distances = torch.cdist(positions,positions)
# print(distances)
sep_distance = .6
mask_close = distances < sep_distance
# Distances will always be a square because it is distance from every agent to every other agent
mask_not_self = ~torch.eye(distances.shape[0],dtype=torch.bool)
mask_separate = mask_close & mask_not_self

for sp in stacked_ps:
  # n x n x a x b
  # Where i, j is sp[i] - sp[j]
  # To steer sp[i] away from sp[j]
  # You can do sp[i] + delta[i][j]
  delta = sp.unsqueeze(1) - sp.unsqueeze(0)
  display(sp)
  print()
  print()
  # display(sp[0]+delta[0])
  print(delta.shape)
  # Sometimes params are n x a x b and sometimes they are just n x a
  # Calculate the norm from the 3rd axis (dim = 2) onward.
  norm_dim = list(range(2,len(delta.shape)))
  norm = delta.norm(dim=norm_dim)
  dimensions_to_add = (None,)*(len(sp.shape)-1)
  norm = norm[(...,)+dimensions_to_add]
  delta_norm = delta/norm
  break

tensor([[[0.2813, 0.6285],
         [0.2049, 0.2254]],

        [[0.2404, 0.5768],
         [0.6771, 0.2962]],

        [[0.0630, 0.6555],
         [0.2056, 0.2023]]])



torch.Size([3, 3, 2, 2])
torch.Size([3, 3, 1, 1])
tensor([[[[    nan,     nan],
          [    nan,     nan]],

         [[ 0.0848,  0.1073],
          [-0.9796, -0.1469]],

         [[ 0.9871, -0.1218],
          [-0.0032,  0.1042]]],


        [[[-0.0848, -0.1073],
          [ 0.9796,  0.1469]],

         [[    nan,     nan],
          [    nan,     nan]],

         [[ 0.3421, -0.1517],
          [ 0.9095,  0.1811]]],


        [[[-0.9871,  0.1218],
          [ 0.0032, -0.1042]],

         [[-0.3421,  0.1517],
          [-0.9095, -0.1811]],

         [[    nan,     nan],
          [    nan,     nan]]]])


In [ ]:
display(scores)
normscores = scores / scores.sum()
display(normscores)
print()
print()
invs = 1 / scores
display(invs)
invnormscores = invs / invs.sum()
display(invnormscores)

tensor([0.5155, 0.9807, 0.7604])

tensor([0.2284, 0.4346, 0.3370])

tensor([1.9400, 1.0197, 1.3150])

tensor([0.4538, 0.2385, 0.3076])

In [ ]:
# This doesn't do moving towards global center but instead moving towards each thing.

In [ ]:
print(f"Num GPUS = {torch.cuda.device_count()}")



print("Example training")
print(f"Final loss = {train_model(optuna.trial.FixedTrial(example_config),display_model=True)}")

In [ ]:
from functools import partial

num_trials = 100

study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.SuccessiveHalvingPruner())
study.optimize(train_model, n_trials=num_trials)

print("DONE")
print("CONFIG:", study.best_trial.params)
print("METRICS:", study.best_trial.values)

In [ ]:
# Need to remember to make submission predictions with logits
all_loader = torch.utils.data.DataLoader(SmokingDataset(train_x_tensor,train_y_tensor), batch_size=bsz)
eval_loader = torch.utils.data.DataLoader(SmokingDataset(eval_x_tensor), batch_size=bsz)

models = make_models(study.best_trial.params)
for epoch in range(study.best_trial.params["epochs"]):
  loss = do_epoch(models,all_loader,all_loader)
  print(loss)

eval_pred = []
with torch.no_grad():
  for batch in eval_loader:
    eval_pred.append(models["model"](batch["data"]))
eval_prob = torch.nn.functional.sigmoid(torch.cat(eval_pred)).squeeze().cpu()
print(f"eval_prob = {eval_prob.shape} w/ final loss = {loss}")

submission = pd.DataFrame({"id" : evaluate.id, "smoking" : eval_prob}).set_index("id")
display(submission)
submission.to_csv(os.path.join(input_dir, "Submissions", "first.csv"))
print("Made submission!")